### IDS 566
### Advanced Text Analytics
### Final Project - The 20 Newsgroup Data Analysis

Submitted by
* Palak Gupta - 677408458
* Ashwyn Muthukumarswami - 675870750
* Gughanraj Selvaraju - 662075168
* Meenakshi Janardanam - 671917004
* Aruna Venkatasubramaniam - 678860489

In [1]:
from __future__ import print_function

import logging
import pandas as pd
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt

from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics

####  Importing the data

In [2]:
data_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
data_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

Evaluating the number of rows in train and test Data

In [3]:
print("%d documents" % len(data_train.filenames))
print("%d categories" % len(data_train.target_names))

print("%d documents" % len(data_test.filenames))
print("%d categories" % len(data_test.target_names))

11314 documents
20 categories
7532 documents
20 categories


In [4]:
# split a training set and a test set
y_train, y_test = data_train.target, data_test.target

## Label Distribution in Training
ls = pd.DataFrame(y_train, columns =['Names'])
ls['Names'].value_counts(normalize=True)

10    0.053032
15    0.052943
8     0.052855
9     0.052766
11    0.052590
13    0.052501
7     0.052501
14    0.052413
5     0.052413
12    0.052236
2     0.052236
3     0.052148
6     0.051706
1     0.051617
4     0.051087
17    0.049850
16    0.048259
0     0.042425
18    0.041100
19    0.033322
Name: Names, dtype: float64

### Cleaning the data to remove numeric and alphanumeric strings

In [6]:
import re
train_text = []
test_text = []
for text in data_train.data:
    train_text.append(re.sub(r'[^a-zA-Z\s]+','',text))
for text1 in data_test.data:
    test_text.append(re.sub(r'[^a-zA-Z\s]+','',text1))

In [7]:
data_train["data"] = train_text
data_test["data"] = test_text

### Stemming the words to get the root word

In [11]:
def stem_words(text):
    stemmer = SnowballStemmer('english')
    return ' '.join([stemmer.stem(word) for word in text.split(' ')])

def lemma_words(text):
    lemmer=WordNetLemmatizer()
    return ' '.join([lemmer.lemmatize(word) for word in text.split(' ')])

Processing for training data

In [30]:
stemmed_text = []
lemmed_text = []
for text in data_train.data:
    stemmed_text.append(stem_words(text))
    lemmed_text.append(lemma_words(text))

In [31]:
data_train["stemmed_data"] = stemmed_text
data_train["lemmed_data"] = lemmed_text

<font color="blue"> We analysed the pre defined structure of the input data from scikit learn library and added the stemmed text as the part of the pre defined structure. This can be ran only once and reused in the multiple stages in the model building. We arrived at this conclusion after experimenting the stemming module to be part of the model building pipeline which increased the model training time exponentially </font>

Processing for test data

In [32]:
stemmed_text = []
lemmed_text = []
for text in data_test.data:
    stemmed_text.append(stem_words(text))
    lemmed_text.append(lemma_words(text))

In [33]:
data_test["stemmed_data"] = stemmed_text
data_test["lemmed_data"] = lemmed_text

##### Creating count Vectorizer

In [34]:
vectorizer = count_vect = CountVectorizer(stop_words='english')
X_train_counts = count_vect.fit_transform(data_train.stemmed_data)
X_test_counts = count_vect.transform(data_test.stemmed_data)
print(X_train_counts.shape)
print(X_test_counts.shape)

(11314, 110877)
(7532, 110877)


##### Creating TFIDF Vectorizer

In [35]:
vectorizer_tfidf = TfidfVectorizer(sublinear_tf=True, max_df = 1,stop_words='english')
X_train_tfidf = vectorizer_tfidf.fit_transform(data_train.stemmed_data)
X_test_tfidf = vectorizer_tfidf.transform(data_train.stemmed_data)
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)

(11314, 65928)
(11314, 65928)


In [36]:
### Getting the total feature vextor length i.e. vocabulary size
feature_names_tfidf = vectorizer_tfidf.get_feature_names()
len(feature_names_tfidf)

65928

## Model 1 - Naive Bayes

Naive bayes with CountVectorizer and unstemmed data

In [37]:
from sklearn.naive_bayes import MultinomialNB
NB = MultinomialNB().fit(X_train_counts, data_train.target)

In [82]:
NB_predicted_train = NB.predict(X_train_counts)
NB_predicted = NB.predict(X_test_counts)

print("Naive Bayes - No Tuning - No Stemming")
print("Training Accuracy:", np.mean(NB_predicted_train == y_train))
print("Training Accuracy:", np.mean(NB_predicted == y_test))

Naive Bayes - No Tuning - No Stemming
Training Accuracy: 0.9695068057274173
Training Accuracy: 0.8108072225172597


In [39]:
# import nltk
# nltk.download()

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)

In [40]:
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

Naive Bayes using pipeling with stemmed data

In [41]:
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect), ('tfidf', TfidfTransformer()), ('mnb', MultinomialNB(fit_prior=False))])

text_mnb_stemmed = text_mnb_stemmed.fit(data_train.data, y_train)

predicted_mnb_stemmed = text_mnb_stemmed.predict(data_test.data)
predicted_mnb_stemmed_train = text_mnb_stemmed.predict(data_train.data)

In [85]:
print("Naive Bayes -No Tuning - Stemming")
print("Training Accuracy:", np.mean(predicted_mnb_stemmed_train == y_train))
print("Test Accuracy:",np.mean(predicted_mnb_stemmed == y_test))


Naive Bayes -No Tuning - Stemming
Training Accuracy: 0.9528018384302633
Test Accuracy: 0.8190387679235263


We observe that the model with stemming and tfidf performes better. For all the models going forwards therefore we used stemmed data and tfidf vectorizer

#### Parameter Tunning - Naive Bayes - using the GridSearch without Stemming

In [62]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-1, 1e-2, 1e-3,1e-4), }

text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(data_train.data, y_train)

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(data_train.data, data_train.target)

C:\Users\palak\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


<font color='blue'> Pipeline and grid search cross validation is performed and paramters(n_jobs = -1) enables us to utilize multiple cores in the system</font> 

In [63]:
gs_clf.best_score_
gs_clf.best_params_

{'clf__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [64]:
predicted_gs_clf = gs_clf.predict(data_test.data)
predicted_gs_clf_train = gs_clf.predict(data_train.data)

In [86]:
print("Naive Bayes -With Tuning - Without Stemming")
print("Training Accuracy:",np.mean(predicted_gs_clf == y_test))
print("Test Accuracy:",np.mean(predicted_gs_clf_train == y_train))

Naive Bayes -With Tuning - Without Stemming
Training Accuracy: 0.8509028146574615
Test Accuracy: 0.9993812975075128


#### Parameter Tunning - Naive Bayes - using the GridSearch with Stemming

In [44]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-1, 1e-2, 1e-3,1e-4), }

text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(data_train.stemmed_data, y_train)

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(data_train.stemmed_data, data_train.target)

C:\Users\palak\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [45]:
### Finding the best score and best parameters
print(gs_clf.best_score_)
print(gs_clf.best_params_)

{'clf__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [94]:
#### Prodiction on test and train data
predicted_gs_clf_train = gs_clf.predict(data_train.stemmed_data)
predicted_gs_clf = gs_clf.predict(data_test.stemmed_data)

In [95]:
print("Naive Bayes -With Tuning - Stemming")
print("Training Accuracy:",np.mean(predicted_gs_clf_train == y_train))
print("Test Accuracy:",np.mean(predicted_gs_clf == y_test))


Naive Bayes -With Tuning - Stemming
Training Accuracy: 0.9992045253668022
Test Accuracy: 0.8315188528943176


## Model - 2 Support Vector Machines

SVM model with without stemming

In [51]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])

text_clf_svm = text_clf_svm.fit(data_train.stemmed_data, data_train.target)
### Predition on test and train data
predicted_svm_train = text_clf_svm.predict(data_train.stemmed_data)
predicted_svm = text_clf_svm.predict(data_test.stemmed_data)


C:\Users\palak\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [89]:

print("Support Vector Machine -No Tuning - No Stemming")
print("Training Accuracy:",np.mean(predicted_svm_train == data_train.target))
print("Test Accuracy:", np.mean(predicted_svm == data_test.target))

Support Vector Machine -No Tuning - No Stemming
Training Accuracy: 0.9613752872547286
Test Accuracy: 0.8158523632501328


#### Parameter Tuning Using SVM - Without Stemming

In [58]:
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])

text_clf_svm = text_clf_svm.fit(data_train.data, y_train)

C:\Users\palak\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [96]:
from sklearn.model_selection import GridSearchCV

parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf-svm__alpha': (1e-1, 1e-2, 1e-3,1e-4 ), }

gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(data_train.data, data_train.target)

### Finding the best score and best parameters
gs_clf_svm.best_score_
gs_clf_svm.best_params_

C:\Users\palak\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\palak\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


{'clf-svm__alpha': 0.0001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [97]:
### Predition on test and train data
glf_predicted_svm_train = gs_clf_svm.predict(data_train.data)
glf_predicted_svm = gs_clf_svm.predict(data_test.data)


In [98]:
print("Support Vector Machine -With Tuning - No Stemming")
print("Training Accuracy:",np.mean(glf_predicted_svm_train == data_train.target))
print("Test Accuracy:",np.mean(glf_predicted_svm == data_test.target))

Support Vector Machine -With Tuning - No Stemming
Training Accuracy: 0.9972600318189854
Test Accuracy: 0.8555496548061604


#### Parameter Tuning Using SVM - Using Stemming

In [53]:
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])

text_clf_svm = text_clf_svm.fit(data_train.stemmed_data, y_train)

C:\Users\palak\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [54]:
from sklearn.model_selection import GridSearchCV

parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf-svm__alpha': (1e-1, 1e-2, 1e-3,1e-4 ), }

gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(data_train.stemmed_data, data_train.target)

### Finding the best score and best parameters
gs_clf_svm.best_score_
gs_clf_svm.best_params_

C:\Users\palak\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\palak\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


{'clf-svm__alpha': 0.0001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [92]:
### Prediction on train and test data
glf_predicted_svm_train = gs_clf_svm.predict(data_train.stemmed_data)
glf_predicted_svm = gs_clf_svm.predict(data_test.stemmed_data)

In [93]:
print("Support Vector Machine -With Tuning - with Stemming")
print("Training Accuracy:",np.mean(glf_predicted_svm_train == data_train.target))
print("Training Accuracy:",np.mean(glf_predicted_svm == data_test.target))

Support Vector Machine -With Tuning - with Stemming
Training Accuracy: 0.9925755700901538
Training Accuracy: 0.8166489644184811


## Model 3  - Logistic Regression with Regularization - Ridge and Lasso

#### Parameter Tuning without Stemming

In [68]:
count_vect = CountVectorizer(stop_words='english',preprocessor=None)
vectorizer_tfidf = TfidfTransformer()

model = Pipeline([("CVectorizer",count_vect),("TF_IDF",vectorizer_tfidf),("SGDC",SGDClassifier(loss='log',max_iter=1000,tol=1e-3))])
parameters = {
    'CVectorizer__max_df':(0.1,),
    'CVectorizer__ngram_range': [(1, 1), (1, 2)],
    'SGDC__penalty':('l1','l2'),
    'SGDC__alpha':(0.1,0.01,0.0001,1e-5,1e-10)
}

In [69]:
GSCV = GridSearchCV(model, parameters,n_jobs=-1)
GSCV.fit(data_train.data,data_train.target)


C:\Users\palak\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('CVectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
 ..._state=None, shuffle=True, tol=0.001,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'CVectorizer__max_df': (0.1,), 'CVectorizer__ngram_range': [(1, 1), (1, 2)], 'SGDC__penalty': ('l1', 'l2'), 'SGDC__alpha': (0.1, 0.01, 0.0001, 1e-05, 1e-10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [70]:
GSCV.best_score_
GSCV.best_params_

{'CVectorizer__max_df': 0.1,
 'CVectorizer__ngram_range': (1, 2),
 'SGDC__alpha': 1e-05,
 'SGDC__penalty': 'l2'}

In [71]:
preds_train = GSCV.predict(data_train.data)
preds = GSCV.predict(data_test.data)

In [72]:
print(np.mean(preds_train == data_train.target))
print(np.mean(preds == data_test.target))

0.9994696835778681
0.8566117896972916


#### Parameter Tuning with Stemming

In [74]:
GSCV = GridSearchCV(model, parameters,n_jobs=-1)
GSCV.fit(data_train.stemmed_data ,data_train.target)

C:\Users\palak\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('CVectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
 ..._state=None, shuffle=True, tol=0.001,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'CVectorizer__max_df': (0.1,), 'CVectorizer__ngram_range': [(1, 1), (1, 2)], 'SGDC__penalty': ('l1', 'l2'), 'SGDC__alpha': (0.1, 0.01, 0.0001, 1e-05, 1e-10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [79]:
GSCV.best_score_
GSCV.best_params_

{'CVectorizer__max_df': 0.1,
 'CVectorizer__ngram_range': (1, 2),
 'SGDC__alpha': 1e-05,
 'SGDC__penalty': 'l2'}

In [77]:
preds_train = GSCV.predict(data_train.stemmed_data)
preds = GSCV.predict(data_test.stemmed_data)

In [78]:
print(np.mean(preds_train == data_train.target))
print(np.mean(preds == data_test.target))

0.9993812975075128
0.8516994158258099
